# **Trabajo practico 1 - Checkpoint 4**
**Grupo**: 26

**Integrantes**:

Garcia, Nicolas

Vallcorba, Agustin

Carbajal Robles, Kevin Emir

## **Librerias y carga del data set**

In [ ]:
!pip install keras==2.12.0
!pip uninstall tensorflow -y
!pip install tensorflow==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.14.0 requires keras<2.15,>=2.14.0, but you have keras 2.12.0 which is incompatible.
Found existing installation: tensorflow 2.14.0
Uninstalling tensorflow-2.14.0:
  Successfully uninstalled tensorflow-2.14.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 44.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate, StratifiedKFold

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import preprocessing

np.random.seed(1)
tf.random.set_seed(1)

Cargamos el dataset de train y test, que es el que obtuvimos en el checkpoint 2.

De igual manera lo haremos para el dataset de test y el de train original, tambien el obtenido del chp1.

In [ ]:
#Dataset de entrenamiento
ds_train = pd.read_csv('sample_data/ds_train_limpio.csv')

#Dataset de test
ds_test = pd.read_csv("sample_data/ds_test_limpio.csv")


#Dataset de test
ds_test_orig = pd.read_csv("sample_data/hotels_test.csv")

#Dataset de entrenamiento chp1
ds_train_chp1 = pd.read_csv('sample_data/ds_train_chp1.csv')

#Dataset de entrenamiento original
ds_train_orig = pd.read_csv("https://drive.google.com/u/0/uc?id=1tjlunFxE63XIpYjWk8SPm2XPxixu8SW4&export=download")

FileNotFoundError: ignored

## **Modelos**

Para entrenar cada modelo, haremos una division del dataset de train.

Entrenaremos con un 70%, y dejaremos un 30% para testear, al igual que el checkpoint 2 y 3.

In [ ]:
#Creo un dataset con los features que voy a usar para clasificar
ds_train_x=ds_train.drop(['is_canceled'], axis='columns', inplace=False)

#Creo un dataset con la variable target
ds_train_y = ds_train['is_canceled'].copy()

#Genero los conjuntos de train y test
x_train, x_test, y_train, y_test = train_test_split(ds_train_x,
                                                    ds_train_y,
                                                    test_size=0.3,  #proporcion 70/30
                                                    random_state=2) #semilla

En todos los modelos, evaluaremos su performance, explicaremos todas las métricas y mostraremos su matriz de confusion.

Definiremos una funcion para calcular las metricas y su matriz de confusion.

In [ ]:
def metricas_y_matriz_confusion(y_prediccion, y_testeo):
  #Calculo las métricas en el conjunto de evaluación
  accuracy=accuracy_score(y_testeo,y_prediccion)
  recall=recall_score(y_testeo,y_prediccion)
  f1=f1_score(y_testeo,y_prediccion,)
  precision=precision_score(y_testeo,y_prediccion)
  cm = confusion_matrix(y_testeo, y_prediccion)

  print("Accuracy: "+str(accuracy))
  print("Recall: "+str(recall))
  print("Precision: "+str(precision))
  print("f1 score: "+str(f1))

  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
  plt.xlabel('Predicción')
  plt.ylabel('Valor Real')
  plt.title('Matriz de Confusión')
  plt.show()

**Modificaciones de datos (OPCIONAL)**

Aca pondriamos alguna que otra modificacion que hariamos en caso de que se nos ocurriesen otras que no habiamos pensado.

En caso de que no usemos esta seccion, hay que borrarla

In [ ]:
#Armo una version escalada con minmax (media=0 , var=1  )
min_max_scaler = preprocessing.MinMaxScaler()
x_train_minmax = min_max_scaler.fit_transform(x_train)
x_test_minmax  = min_max_scaler.transform(x_test)

In [ ]:
#Armo una version estandarizada
stand_scaler = preprocessing.StandardScaler()
x_train_norm = stand_scaler.fit_transform(x_train)
x_test_norm = stand_scaler.transform(x_test)


In [ ]:
#Armo una version escalada con minmax (media=0 , var=1  )
ds_test_minmax = min_max_scaler.fit_transform(ds_test)

In [ ]:
ds_test_norm=stand_scaler.transform(ds_test)

In [ ]:
def f1_metric(y_true, y_pred):
    f1 = tf.py_function(f1_score, (y_true, tf.round(y_pred)), tf.float32)
    return f1

### **Modelo 1**

Red neuronal basica, en donde usaremos los datos transformados, ya sean de **MinMax** o **Normalizados**.

In [ ]:
# calcula la cantidad de clases
cant_clases=len(np.unique(y_train))
d_in=(x_train_minmax.shape[1])

modelo = keras.Sequential([
    keras.layers.Dense(64, input_shape=(d_in,), activation='relu'),  # Capa densa con 64 unidades y función de activación ReLU
    keras.layers.Dense(32, activation='relu'),  # Capa densa con 32 unidades y función de activación ReLU
    keras.layers.Dense(16, activation='relu'),  # Capa densa con 32 unidades y función de activación ReLU
    keras.layers.Dense(8, activation='relu'),  # Capa densa con 32 unidades y función de activación ReLU
    keras.layers.Dense(1, activation='sigmoid')  # Capa de salida con función de activación sigmoid
    ])

modelo.summary()

In [ ]:
modelo.compile(
              optimizer=keras.optimizers.SGD(learning_rate=0.01),
              loss='binary_crossentropy',
  # metricas para ir calculando en cada iteracion o batch
              metrics=[f1_metric],
)

cant_epochs=100
modelo_entrenando = modelo.fit(x_train_minmax,y_train,
                               epochs=cant_epochs,
                               batch_size=50,
                               verbose=False)

In [ ]:
y_predic_ej1 = modelo.predict(x_test_minmax)
y_predic_cat_ej1 = np.where(y_predic_ej1>0.4,1,0)

ds_validacion=pd.DataFrame(y_predic_cat_ej1,y_test).reset_index()
ds_validacion.columns=['y_pred','y_real']

metricas_y_matriz_confusion(ds_validacion['y_pred'], ds_validacion['y_real'])

### **Modelo 2**

In [ ]:
cant_clases = 1
d_in = x_train_minmax.shape[1]

modelo2 = keras.Sequential([
    keras.layers.Dense(128, input_shape=(d_in,), activation='relu'),  # Capa densa con 128 unidades y función de activación ReLU
    keras.layers.Dropout(0.2),  # Capa de dropout con 20% de dropout
    keras.layers.Dense(64, activation='relu'),  # Capa densa con 64 unidades y función de activación ReLU
    keras.layers.Dropout(0.2),  # Capa de dropout con 20% de dropout
    keras.layers.Dense(32, activation='relu'),  # Capa densa con 32 unidades y función de activación ReLU
    keras.layers.Dense(1, activation='sigmoid')  # Capa de salida con función de activación sigmoid
])

modelo2.summary()

In [ ]:
modelo2.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),  # Optimizador Adam con learning rate 0.001
    loss='binary_crossentropy',  # Pérdida para clasificación binaria
    metrics=['accuracy']  # Métrica de precisión
)

cant_epochs = 50
batch_size = 64

modelo2_entrenando = modelo2.fit(x_train_minmax, y_train, epochs=cant_epochs, batch_size=batch_size, validation_data=(x_test_minmax, y_test))

In [ ]:
y_predic_ej2 = modelo2.predict(x_test_minmax)
y_predic_cat_ej2 = np.where(y_predic_ej2>0.5,1,0)

ds_validacion2=pd.DataFrame(y_predic_cat_ej2,y_test).reset_index()
ds_validacion2.columns=['y_pred','y_real']

metricas_y_matriz_confusion(ds_validacion2['y_pred'], ds_validacion2['y_real'])

### **Modelo 3**

In [ ]:
cant_clases = 1
d_in = x_train_minmax.shape[1]

modelo3 = keras.Sequential([
    keras.layers.Dense(256, input_shape=(d_in,), activation='tanh'),  # Capa densa con 256 unidades y función de activación tanh
    keras.layers.Dropout(0.3),  # Capa de dropout con 30% de dropout
    keras.layers.Dense(128, activation='tanh'),  # Capa densa con 128 unidades y función de activación tanh
    keras.layers.Dropout(0.3),  # Capa de dropout con 30% de dropout
    keras.layers.Dense(64, activation='tanh'),  # Capa densa con 64 unidades y función de activación tanh
    keras.layers.Dense(1, activation='sigmoid')  # Capa de salida con función de activación sigmoid
])

modelo3.summary()

In [ ]:
modelo3.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),  # Optimizador Adam con learning rate 0.001
    loss='binary_crossentropy',  # Pérdida para clasificación binaria
    metrics=['accuracy']  # Métrica de precisión
)

cant_epochs = 50
batch_size = 64

modelo3_entrenando = modelo3.fit(x_train_minmax, y_train, epochs=cant_epochs, batch_size=batch_size, validation_data=(x_test_minmax, y_test))

In [ ]:
y_predic_ej3 = modelo3.predict(x_test_minmax)
y_predic_cat_ej3 = np.where(y_predic_ej3>0.5,1,0)

ds_validacion3=pd.DataFrame(y_predic_cat_ej3,y_test).reset_index()
ds_validacion3.columns=['y_pred','y_real']

metricas_y_matriz_confusion(ds_validacion3['y_pred'], ds_validacion3['y_real'])

## **Prediccion final**

Seleccionamos el dataset a predecir.

In [ ]:
x_submission = de_test_minmax

Luego de entrenar estos modelos con el dataset de train, vamos a predecir con el test.

In [ ]:
y_pred_final = modelo3.predict(x_submission)

In [ ]:
y_pred_final_2d = y_pred_final.flatten()

In [ ]:
y_pred_final_final = np.where(y_pred_final_2d>0.5,1,0)

## **Submission**

Haremos la exportacion de esta prediccion para subirla a la competencia de **kaggle**.

Cargamos el dataset final con las predicciones hechas.

In [ ]:
df_submissing = pd.DataFrame({'id': ds_test_orig['id'], 'is_canceled': y_pred_final_final.astype(int)})

In [ ]:
df_submissing.head()

Finalmente, exportamos.

In [ ]:
df_submissing.to_csv('./submission_chp4_4.csv', index=False)